In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/BraTS2021_00495.tar
/kaggle/input/BraTS2021_Training_Data.tar
/kaggle/input/BraTS2021_00621.tar


In [3]:
import tensorflow as tf
# detect and init the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    # instantiate a distribution strategy
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
except:
    pass
%pip install glob
%pip install nibabel

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import tarfile
file = tarfile.open('/kaggle/input/BraTS2021_Training_Data.tar')

file.extractall('./BraTS2021_Training_Data')
file.close()
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float32')
mixed_precision.set_global_policy(policy)
tf.keras.backend.set_floatx('float32')

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages are logged (default behavior)
                                          # 1 = INFO messages are not printed
                                          # 2 = INFO and WARNING messages are not printed
                                          # 3 = INFO, WARNING, and ERROR messages are not printed
import tensorflow as tf
tf.config.run_functions_eagerly(False)
tf.get_logger().setLevel('ERROR')


In [26]:
import tensorflow as tf
import tensorflow.keras as kr
import tensorflow.keras as keras
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import keras
import random
import time
import math



folder_path = [os.path.join(r"BraTS2021_Training_Data", i) for i in os.listdir(r"BraTS2021_Training_Data")]
imgs = []


for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])



#input directory and output directory
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
x_train_path = inputpath[:1126].copy()
y_train_path = outputpath[:1126].copy()
x_eval_path = inputpath[1126:1189].copy()
y_eval_path = outputpath[1126:1189].copy()
x_test_path = inputpath[1189:].copy()
y_test_path = outputpath[1189:].copy()  

parition = {"train": x_train_path, "validation": x_eval_path}
label = {"train": y_train_path, "validation": y_eval_path}



#set up var
x_VolSlice_start = 10
x_VolSlice_end = 150

y_VolSlice_start = 10
y_VolSlice_end = 150


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------model making--------------------------------------------------------------------------------



import tensorflow as tf

class GradientAccumulationModel(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_gradients = tf.Variable(0, dtype=tf.int32, trainable=False)
        self.n_acum_steps = tf.Variable(4, dtype=tf.int32, trainable=False)
        self.gradient_accumulation = [tf.Variable(tf.zeros_like(v, dtype=tf.float32), trainable=False) for v in self.trainable_variables]

    def train_step(self, data):
        self.n_gradients.assign_add(1)
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        
        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign_add(gradients[i])
        
        tf.cond(tf.equal(self.n_gradients, self.n_acum_steps), 
                lambda: self.apply_accu_gradients(),
                lambda: tf.constant(0))
        
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def apply_accu_gradients(self):
        self.optimizer.apply_gradients(zip(self.gradient_accumulation, self.trainable_variables))
        
        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign(tf.zeros_like(self.trainable_variables[i], dtype=tf.float32))
        
        self.n_gradients.assign(0)
        return tf.constant(1)

#note:
# conv3d need a 5d tensor (height,width,depth,channels)


import tensorflow as tf
from tensorflow.keras import layers, Model
# dice loss as defined above for 4 classes
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


 
# These functions are used for evaluating the performance of a segmentation model on three different classes
# in medical imaging (presumably related to brain tumor segmentation).
# Input Parameters:
# y_true: The ground truth segmentation mask for the edema class.
# y_pred: The predicted segmentation mask for the edema class.
# epsilon: A small constant to avoid division by zero.


import tensorflow as tf
from tensorflow import keras
K = keras.backend

def dice_coef(y_true, y_pred, smooth=1.):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Update other metric functions similarly
def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)

# Update other metric functions if necessary
def precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def sensitivity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras import layers, Model

class PadCropLayer(layers.Layer):
    def __init__(self, target_shape, **kwargs):
        super(PadCropLayer, self).__init__(**kwargs)
        self.target_shape = target_shape

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        paddings = []
        for i in range(1, 4):  # For H, W, D dimensions
            diff = self.target_shape[i] - input_shape[i]
            pad_before = diff // 2
            pad_after = diff - pad_before
            paddings.append([tf.maximum(0, pad_before), tf.maximum(0, pad_after)])
        paddings = [[0, 0]] + paddings + [[0, 0]]  # Add padding for batch and channel dimensions
        
        padded = tf.pad(inputs, paddings)
        
        # Crop if necessary
        cropped = padded[:, :self.target_shape[1], :self.target_shape[2], :self.target_shape[3], :]
        
        return cropped

def conv_block_3d(input_tensor, num_filters, kernel_size=3, dropout_rate=0.3):
    x = layers.Conv3D(num_filters, kernel_size, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Conv3D(num_filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_3d(input_shape=(156, 156, 140, 4), num_classes=4):
    inputs = layers.Input(input_shape)
    
    # Encoder (downsampling)
    conv1 = conv_block_3d(inputs, 32)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = conv_block_3d(pool1, 64)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = conv_block_3d(pool2, 128)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = conv_block_3d(pool3, 256)
    pool4 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
    # Bridge
    conv5 = conv_block_3d(pool4, 512)
    
    # Decoder (upsampling)
    up6 = layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv5)
    up6 = PadCropLayer(tf.shape(conv4))(up6)
    concat6 = layers.Concatenate()([up6, conv4])
    conv6 = conv_block_3d(concat6, 256)
    
    up7 = layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv6)
    up7 = PadCropLayer(tf.shape(conv3))(up7)
    concat7 = layers.Concatenate()([up7, conv3])
    conv7 = conv_block_3d(concat7, 128)
    
    up8 = layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv7)
    up8 = PadCropLayer(tf.shape(conv2))(up8)
    concat8 = layers.Concatenate()([up8, conv2])
    conv8 = conv_block_3d(concat8, 64)
    
    up9 = layers.Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='same')(conv8)
    up9 = PadCropLayer(tf.shape(conv1))(up9)
    concat9 = layers.Concatenate()([up9, conv1])
    conv9 = conv_block_3d(concat9, 32)
    
    outputs = layers.Conv3D(num_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)

# Define loss function
def weighted_categorical_crossentropy(class_weights):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        weighted_losses = y_true * tf.expand_dims(class_weights, 0) * tf.math.log(y_pred + 1e-7)
        return -tf.reduce_sum(weighted_losses, axis=-1)
    return loss

def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=[1,2,3,4])
    denominator = tf.reduce_sum(y_true + y_pred, axis=[1,2,3,4])
    return 1 - numerator / (denominator + tf.keras.backend.epsilon())

def combined_loss(y_true, y_pred):
    class_weights = tf.constant([0.1, 1.0, 1.0, 1.0])
    wce = weighted_categorical_crossentropy(class_weights)(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return wce + dice



# Define callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.001 * (0.1 ** (epoch // 30))
)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "3D-UNet-weights-{epoch:02d}-{val_accuracy:.3f}.keras",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
csv_logger = tf.keras.callbacks.CSVLogger('training_3D_UNet.log')

# Train the model


# Example class weights (adjust based on your data)
class_weights = tf.constant([0.1, 1.0, 1.0, 1.0])

try:
    with tpu_strategy.scope():
        model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
        model.compile(
    loss=combined_loss,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=4),
             dice_coefficient, precision, sensitivity, specificity,
             dice_coef_necrotic, dice_coef_edema, dice_coef_enhancing]
)
    tf.debugging.set_log_device_placement(True)
except:
    model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
    model.compile(
    loss=combined_loss,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=4),
             dice_coefficient, precision, sensitivity, specificity,
             dice_coef_necrotic, dice_coef_edema, dice_coef_enhancing]
)
    tf.debugging.set_log_device_placement(True)
    
model.summary()





# In your __data_generation method, after loading and preprocessing:


#datagen
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, labels, batch_size=5, dim=(156,156,140), n_channels=4,
                 n_classes=4, shuffle=True, real_batchsize_custom=2, frames_chunk=18,
                 y_VolSlice_start=0, y_VolSlice_end=None, **kwargs):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.real_batchsize_custom = real_batchsize_custom
        self.frames_chunk = frames_chunk
        self.y_VolSlice_start = y_VolSlice_start
        self.y_VolSlice_end = y_VolSlice_end if y_VolSlice_end is not None else dim[2]
        super().__init__(**kwargs)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = np.empty((self.real_batchsize_custom, *self.dim, self.n_channels))
        y = np.empty((self.real_batchsize_custom, *self.dim, self.n_classes), dtype=float)

        initial_point = 0
        final_point = self.frames_chunk
        for x in range(self.real_batchsize_custom):    
            indexes_orig = indexes[initial_point:final_point]
            list_IDs_temp = [self.list_IDs[k] for k in indexes_orig]
            X_in, y_in = self.__data_generation(list_IDs_temp)
            X[x] = X_in
            y[x] = y_in
            initial_point = final_point
            final_point = final_point + self.frames_chunk   
        return X, y

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        X = np.empty((1, *self.dim, self.n_channels))
        y = np.zeros((1, *self.dim, self.n_classes))
        
        for i, file_paths in enumerate(list_IDs_temp):
            image = self.load_nifti(file_paths)
            X[0] = image
            
            seg = nib.load(self.labels[i]).get_fdata().astype(np.float16)
            seg = self.preprocess_mask(seg)
            
            y[0] = seg
        
        X = X.astype(np.float16)
        y = y.astype(np.float16)
        X[0], y[0] = self.augment(X[0], y[0])
        X_norm = self.normalize(X[0])
        return X_norm, y[0]

    def load_nifti(self, file_paths):
        image = np.empty((*self.dim, self.n_channels))
        for channel, file_path in enumerate(file_paths):
            img = nib.load(file_path).get_fdata().astype(np.float16)
            img = self.preprocess_image(img)
            image[..., channel] = img
        return image

    def preprocess_image(self, img):
        # Resize to match model input dimensions
        return resize(img, self.dim, order=1, preserve_range=True)

    def preprocess_mask(self, mask):
        mask[mask==4] = 3
        mask = resize(mask, self.dim, order=0, preserve_range=True)
        return tf.one_hot(mask.astype(int), self.n_classes)

    def normalize(self, image):
        for c in range(image.shape[-1]):
            channel = image[..., c]
            min_val = np.min(channel)
            max_val = np.max(channel)
            if max_val > min_val:
                image[..., c] = (channel - min_val) / (max_val - min_val)
        return image

    def augment(self, image, mask):
        # Implement data augmentation here
        return image, mask

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
            
            
            
training_generator = DataGenerator(parition["train"], label["train"], dim=(156, 156, 140), n_channels=4, n_classes=4)
valid_generator = DataGenerator(parition["validation"], label["validation"], dim=(156, 156, 140), n_channels=4, n_classes=4)


#model fitting


from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

filepath="3D-UNet-2018-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.keras" 

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

csv_logger = CSVLogger('training_2021_2D_UNet.log')

history = model.fit(
    training_generator,
    epochs=100,
    steps_per_epoch=len(parition["train"]),
    validation_data=valid_generator,
    validation_steps=len(parition["validation"]),
    callbacks=[checkpoint, early_stop, csv_logger, lr_schedule])

with open('history.txt', 'w') as f:
    f.write(history)
try:
    model.save("final_model.h5")
except:
    model.save("final_model.keras")



Done
['BraTS2021_Training_Data/BraTS2021_00622/BraTS2021_00622_t1.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00622/BraTS2021_00622_t1ce.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00622/BraTS2021_00622_t2.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00622/BraTS2021_00622_flair.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00622/BraTS2021_00622_seg.nii.gz']


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


model

In [ ]:
model